In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import conv2d,max_pool2d,log_softmax,linear,relu,dropout2d
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import matplotlib.image as img
from PIL import Image, ImageDraw
import numpy as np
import os
import pandas as pd
import cv2


In [3]:
train_dataset="/content/drive/MyDrive/jackfruit_mango_classifier/training_new_trial"
test_dataset="/content/drive/MyDrive/jackfruit_mango_classifier/test_set"
valid_dataset="/content/drive/My Drive/jackfruit_mango_classifier/validation"
train_set ="/content/drive/MyDrive/jackfruit_mango_classifier/training_new_trial"

In [4]:
#data = pd.read_csv(r'/content/drive/MyDrive/jackfruit_mango_classifier/combined/training_data.csv')
#data.head()
#data.tail()

typing out code to get label auto matically use when required by copying to code cell
dataset = ImageFolder(r'/content/drive/MyDrive/jackfruit_mango_classifier/training_set/training_data.csv',transform = ToTensor)
img,label= dataset[0]
print(dataset.classes)

In [5]:
#typing out code to get label auto matically use when required by copying to code cell
mean = torch.tensor([0.485, 0.456, 0.406], dtype=torch.float)
std = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float)
transform1 = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)])
train_data = ImageFolder(r'/content/drive/MyDrive/jackfruit_mango_classifier/training_new_trial/',transform = transform1)
img,label= train_data[1]
print(img.shape)
print(train_data[0])
print(train_data.classes)

torch.Size([3, 32, 32])
(tensor([[[ 0.2282, -0.0629, -0.2342,  ...,  1.5468,  1.5810,  1.8037],
         [ 0.0912, -0.1999, -0.2171,  ...,  1.1187,  1.1015,  1.3927],
         [ 0.1254, -0.2342, -0.1657,  ...,  0.7591,  0.7762,  1.1187],
         ...,
         [ 1.3242, -0.2856, -0.9020,  ...,  1.1529,  1.1529,  1.4269],
         [ 0.9303,  0.3309, -0.8678,  ...,  0.6734,  0.4508,  1.4269],
         [ 0.2624, -0.1999,  0.3652,  ..., -0.2342, -0.4397,  1.2557]],

        [[ 0.3102,  0.0301, -0.1800,  ...,  1.4307,  1.1506,  1.6583],
         [ 0.2052, -0.0749, -0.1625,  ...,  0.9055,  0.5553,  1.0455],
         [ 0.1877, -0.1275, -0.1275,  ...,  0.4678,  0.1527,  0.6604],
         ...,
         [ 1.3431, -0.2150, -0.8452,  ...,  1.4482,  1.4832,  1.6232],
         [ 0.9930,  0.3803, -0.8803,  ...,  0.9405,  0.7829,  1.6232],
         [ 0.3277, -0.1625,  0.3627,  ..., -0.1975, -0.3025,  1.4657]],

        [[ 0.8448,  0.5659,  0.3045,  ...,  1.2108,  0.5834,  1.4897],
         [ 0.7228,  

In [6]:
valid_data = ImageFolder(r'/content/drive/MyDrive/jackfruit_mango_classifier/validation/',transform = transform1)
img,label= valid_data[0]
print(valid_data[0])
print(valid_data.classes)
print(img.shape,img.type)

(tensor([[[-1.7754, -1.8782, -1.5870,  ..., -0.5082, -0.2856, -0.4911],
         [-1.6555, -1.7240, -1.5014,  ..., -0.7650, -0.5596, -0.5767],
         [-1.6384, -1.7754, -1.6555,  ..., -0.7137, -0.4226, -0.3198],
         ...,
         [-1.4329, -1.4843, -1.5699,  ..., -0.5596, -0.9363, -1.2445],
         [-1.5699, -1.6042, -1.6727,  ..., -0.4226, -0.9705, -1.1418],
         [-1.6213, -1.6898, -1.7069,  ..., -0.4226, -1.0904, -1.0390]],

        [[-1.8431, -1.8957, -1.6681,  ..., -0.6527, -0.4251, -0.6176],
         [-1.7381, -1.7906, -1.6506,  ..., -0.8452, -0.6352, -0.6527],
         [-1.7381, -1.8431, -1.7906,  ..., -0.8452, -0.5126, -0.4251],
         ...,
         [-1.5805, -1.6155, -1.6681,  ..., -0.7927, -1.1604, -1.3880],
         [-1.6681, -1.6856, -1.7206,  ..., -0.6877, -1.2129, -1.3179],
         [-1.6856, -1.7381, -1.7556,  ..., -0.6877, -1.2654, -1.1779]],

        [[-1.7522, -1.7696, -1.6650,  ..., -0.7936, -0.5844, -0.7587],
         [-1.6999, -1.7173, -1.6650,  ..., -

In [7]:
dataset = torchvision.datasets.ImageFolder(train_set, transform=transform1)

# Create a Train DataLoader using Train Dataset
train_data_loader = torch.utils.data.DataLoader(
    dataset=train_data,
    batch_size=5,
    shuffle=False,
    #num_workers=0
)
# Create a Test DataLoader using Test Dataset
test_data_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=5,
    shuffle=False,
    #num_workers=0
)
val_data_loader = torch.utils.data.DataLoader(
    dataset = valid_data, 
    batch_size = 5, 
    shuffle=False, 
    #num_workers=0
    )

In [16]:
#architecture based on Lenet
class newmodel(nn.Module):
  #defining basic layers
  def __init__(self):
    super(newmodel, self).__init__()
    self.conv1=nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
    self.pool= nn.MaxPool2d(2,2)
    self.conv2=nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
    self.fully_connected1=nn.Linear(400,120)
    self.fully_connected2 =nn.Linear(120,84)
    self.fully_connected3=nn.Linear(84,2)
    self.drop=nn.Dropout2d()
    

  def forward(self,x):
    x=self.pool(relu(self.conv1(x)))
    x=self.pool(relu(self.conv2(x)))
  # print(x.shape)
    x = x.view(x.shape[0],-1) 
  #  print(x.shape)
    x=relu(self.fully_connected1(x))
    x=relu(self.fully_connected2(x))
    #x=self.drop(x)
    x=((self.fully_connected3(x))) 
    return x


In [17]:
model = newmodel()
print(model)
model_save_name = 'jango_classifier.pt'
path_model = "/content/drive/MyDrive/jackfruit_mango_classifier/" + model_save_name
model.load_state_dict(torch.load(path_model))
print(model)

newmodel(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fully_connected1): Linear(in_features=400, out_features=120, bias=True)
  (fully_connected2): Linear(in_features=120, out_features=84, bias=True)
  (fully_connected3): Linear(in_features=84, out_features=2, bias=True)
  (drop): Dropout2d(p=0.5, inplace=False)
)
newmodel(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fully_connected1): Linear(in_features=400, out_features=120, bias=True)
  (fully_connected2): Linear(in_features=120, out_features=84, bias=True)
  (fully_connected3): Linear(in_features=84, out_features=2, bias=True)
  (drop): Dropout2d(p=0.5, inplace=False)
)


In [18]:
#model = newmodel()
print(model)
if torch.cuda.is_available():
    model.cuda()
else:
    model.cpu()

newmodel(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fully_connected1): Linear(in_features=400, out_features=120, bias=True)
  (fully_connected2): Linear(in_features=120, out_features=84, bias=True)
  (fully_connected3): Linear(in_features=84, out_features=2, bias=True)
  (drop): Dropout2d(p=0.5, inplace=False)
)


In [19]:
#loss function and optimiser
epochs = 100
batch_size = 5
learning_rate = 0.01
loss_f = torch.nn.NLLLoss()
loss_func =nn.CrossEntropyLoss()
optimise=torch.optim.Adam(model.parameters(),lr=0.01)
#loss_func=loss_func.to(device)


In [20]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [ ]:
#training and testing
train_loss=[]
val_loss=[]
accuracy = []

for epoch in range(epochs):
  trainloss=0
  validloss=0

  model.train()
  for data,label in train_data_loader:
    #print(data.shape)
    data = data.to(device)
    label = label.to(device)
    #print(label.shape)
    #to clear gradients
    optimise.zero_grad()
    output = model(data)
   # print(output)
    loss = loss_func(output,label)
    loss.backward()
    optimise.step()
    trainloss += loss.item() #.item to convert to float
    train_loss.append(trainloss)
    #accuracy.append((label==np.argmax(output)).sum().item() / label.shape[0])

    model.eval()

  for data,label in val_data_loader:
    data = data.to(device)
    label = label.to(device)
    output = model(data)
    loss = loss_func(output,label)
    validloss += loss.item() 
    val_loss.append(validloss)
    accuracy.append((label==torch.argmax(output.data,1)).sum().item() / label.shape[0])

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \t valid_accuracy: {:.2f}'.format(epoch, trainloss, validloss,sum(accuracy)/len(accuracy)))
    
  torch.save(model.state_dict(), '/content/drive/MyDrive/jackfruit_mango_classifier/model.ckpt')
     

Epoch: 0 	Training Loss: 78.121905 	Validation Loss: 3.184111 	 valid_accuracy: 0.50
Epoch: 1 	Training Loss: 80.586682 	Validation Loss: 3.307491 	 valid_accuracy: 0.50
Epoch: 2 	Training Loss: 81.752412 	Validation Loss: 3.349154 	 valid_accuracy: 0.50
Epoch: 3 	Training Loss: 82.157848 	Validation Loss: 3.363000 	 valid_accuracy: 0.50
Epoch: 4 	Training Loss: 82.296102 	Validation Loss: 3.367737 	 valid_accuracy: 0.50
Epoch: 5 	Training Loss: 82.345603 	Validation Loss: 3.369499 	 valid_accuracy: 0.50
Epoch: 6 	Training Loss: 82.365671 	Validation Loss: 3.370272 	 valid_accuracy: 0.50
Epoch: 7 	Training Loss: 82.375648 	Validation Loss: 3.370697 	 valid_accuracy: 0.50
Epoch: 8 	Training Loss: 82.381877 	Validation Loss: 3.370985 	 valid_accuracy: 0.50
Epoch: 9 	Training Loss: 82.386449 	Validation Loss: 3.371208 	 valid_accuracy: 0.50
Epoch: 10 	Training Loss: 82.390135 	Validation Loss: 3.371390 	 valid_accuracy: 0.50
Epoch: 11 	Training Loss: 82.393211 	Validation Loss: 3.371546 	

In [ ]:
model.eval()
with torch.no_grad():
  correct=0
  total=0
  for data,label in val_data_loader:
    data = data.to(device)
    label = label.to(device)
    print("label=",label)

    output = model(data)
    print("output",output.data)
    predicted = torch.argmax(output.data,1)
    print("chech",label,predicted,total,"iiii")
    total+=1
    if (predicted==label):
        correct+=1
  print('Accuracy:'+ str(100*correct/total))
  #return data,predict

In [ ]:
#saving trained model to drive
model_save_name = 'jango_classifier.pt'
path_model = "/content/drive/MyDrive/jackfruit_mango_classifier/" + model_save_name
torch.save(model.state_dict(), path_model)

In [ ]:
#visualise fitting
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.plot(train_loss, label='Training loss')
plt.plot(val_loss, label='Validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(frameon=False)


In [ ]:
#loading model from drive
#model.load_state_dict(torch.load(path_model))

In [ ]:
#gettimg images from url
urll = "https://thumbs.dreamstime.com/b/fruits-mango-scientific-name-mangifera-indica-anacardiaceae-ripened-fruit-piled-up-sale-thiruvananthapuram-kerala-india-48649430.jpg"
from imageio import imread
image = imread(urll)
image = Image.fromarray(image)
plt.imshow(image)

In [ ]:

# model = torch.load('BasicCNN_Pytorch/model_50.pth', map_location=device) 
# model.eval()

class_list ={0:'chakka', 1:'maanga'}
detransform= transforms.Compose([
    transforms.Normalize(mean = -mean/std, std = 1./std),
    transforms.ToPILImage()
])
transform1 = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)])



with torch.no_grad():
        image=transform1(image)
        output = model(image)
        output = nn.Softmax(dim=1)(output)[0]*100
        
        id = output.argmax().data.item()
        oclass = list(class_list.keys())[id]
        output = output.int().data.cpu().numpy()

        display(detransform(image))
        print(class_list[oclass], ':', output[id], '%', '\n\n')

model.train()
pass